In [203]:
import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv

In [204]:
load_dotenv()
API_KEY = os.getenv('OPENROUTER_API_KEY')

In [205]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

### Initialization

In [206]:
llm = ChatOpenAI(
    openai_api_key=API_KEY,
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0.5,
)

### Response JSON format

In [207]:
RESPONSE_JSON = {
  "1": {
    "no": "1",
    "mcq": "Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "correct": "correct ans"
  },
  "2": {
    "no": "2",
    "mcq": "Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "correct": "correct ans"
  },
  "3": {
    "no": "3",
    "mcq": "Multiple choice question.",
    "options": {
      "A": "Choice A",
      "B": "Choice B",
      "C": "Choice C",
      "D": "Choice D"
    },
    "correct": "correct ans"
  }
}


### 1st Template (Generation)

In [208]:
TEMPLATE = """
You are an expert question setter with deep knowledge of the subject: {subject}. 
Generate {number} multiple choice questions (MCQs) from the following text. Use a {tone} tone while creating the questions.

TEXT:
{text}

INSTRUCTIONS:
- Each question should have exactly 4 options: A, B, C, and D.
- Only one option should be correct.
- Vary difficulty from easy to hard and cover different parts of the text.
- Ensure relevance and factual accuracy.
- Do not include explanations, code fences, or any extra text.

RESPONSE FORMAT should be exactly as:
{response_json} without any exra information
"""


In [209]:
mcq_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    template=TEMPLATE
)

### 1st Chain

In [210]:
generation_chain = LLMChain(
    llm=llm,
    prompt=mcq_generation_prompt,
    output_key="quiz",
    verbose=True
)

### 2nd Template (review)

In [211]:
TEMPLATE2 = """
You are an expert in English grammar and education, skilled at analyzing and adjusting multiple choice questions (MCQs) for clarity, tone, and student ability.

Subject: {subject}

Below is a JSON array of generated MCQs:
{quiz}

INSTRUCTIONS:
- Check and correct all grammar.
- Ensure each question’s complexity matches the students’ level.
- Maintain relevance to the subject.
- Preserve the specified tone carefully: {tone}.
- If any question is too complex, unclear, or off-tone, revise it while keeping its original intent.

RESPONSE FORMAT should be exactly same as {response_json} without any extra text:
"""


In [212]:
mcq_evaluation_prompt = PromptTemplate(
    input_variables=['quiz', 'subject', 'tone', 'response_json'],
    template=TEMPLATE2
)

### 2nd Chain

In [213]:
review_chain = LLMChain(
    llm=llm,
    prompt=mcq_evaluation_prompt,
    output_key="reviewed_quiz",
    verbose=True
)

### Combining 1st and 2nd Chains

In [214]:
generate_evaluate_chain = SequentialChain(
    chains=[generation_chain, review_chain],
    input_variables=['text', 'number', 'subject', 'tone', 'response_json'],
    output_variables=['quiz', 'reviewed_quiz'], 
    verbose=True  
)


### Reading my source data

In [215]:
with open("oop_info.txt", "r", encoding="utf-8") as file:
    source_text = file.read()

In [216]:
source_text

"Object-Oriented Programming (OOP) is a programming paradigm that is based on the concept of objects, which can contain both data in the form of fields (often known as attributes or properties) and methods (functions or procedures) that operate on the data. In OOP, the focus is on organizing code around objects and the interactions between them, rather than on logic and functions. OOP allows programmers to model real-world entities and their relationships, which makes it easier to design, manage, and scale software systems.\n\nA class in OOP is a blueprint for creating objects. It defines the properties (data attributes) and methods (functions) that the objects created from the class will have. The class itself does not hold any data, but rather it provides a template from which individual objects are created. Each object is an instance of a class, and each object can have its own specific values for the attributes defined in the class. This allows objects to represent real-world entit

### Serializing Python dictionary to JSON-formatted-string

In [217]:
json_string = json.dumps(RESPONSE_JSON)
print(json_string)

{"1": {"no": "1", "mcq": "Multiple choice question.", "options": {"A": "Choice A", "B": "Choice B", "C": "Choice C", "D": "Choice D"}, "correct": "correct ans"}, "2": {"no": "2", "mcq": "Multiple choice question.", "options": {"A": "Choice A", "B": "Choice B", "C": "Choice C", "D": "Choice D"}, "correct": "correct ans"}, "3": {"no": "3", "mcq": "Multiple choice question.", "options": {"A": "Choice A", "B": "Choice B", "C": "Choice C", "D": "Choice D"}, "correct": "correct ans"}}


### Creating Response and Tracking Token Usage

In [218]:
NUMBER = 5
SUBJECT = 'Object Oriented Programming'
TONE = 'Simple'

In [219]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text" : source_text,
            "number" : NUMBER,
            'subject' : SUBJECT,
            'tone' : TONE,
            'response_json' : json_string
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert question setter with deep knowledge of the subject: Object Oriented Programming. 
Generate 5 multiple choice questions (MCQs) from the following text. Use a Simple tone while creating the questions.

TEXT:
Object-Oriented Programming (OOP) is a programming paradigm that is based on the concept of objects, which can contain both data in the form of fields (often known as attributes or properties) and methods (functions or procedures) that operate on the data. In OOP, the focus is on organizing code around objects and the interactions between them, rather than on logic and functions. OOP allows programmers to model real-world entities and their relationships, which makes it easier to design, manage, and scale software systems.

A class in OOP is a blueprint for creating objects. It defines the properties (data attributes) and methods (functions) that the objects create

In [220]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Input Tokens: {cb.prompt_tokens}")      
print(f"Output Tokens: {cb.completion_tokens}")
print(f"Total Costs: {cb.total_cost}")


Total Tokens: 2312
Input Tokens: 1613
Output Tokens: 699
Total Costs: 0.0


In [221]:
quiz = response.get('quiz')
quiz

'{"1": {"no": "1", "mcq": "What is the primary focus of Object-Oriented Programming?", "options": {"A": "Logic and functions", "B": "Objects and their interactions", "C": "Data structures and algorithms", "D": "Procedural programming"}, "correct": "B"}, "2": {"no": "2", "mcq": "What does a class in OOP define?", "options": {"A": "A specific object with its own data", "B": "Properties and methods for objects created from it", "C": "A set of unrelated functions", "D": "A collection of abstract data types"}, "correct": "B"}, "3": {"no": "3", "mcq": "Which OOP concept involves hiding implementation details and exposing only essential features?", "options": {"A": "Encapsulation", "B": "Inheritance", "C": "Polymorphism", "D": "Abstraction"}, "correct": "D"}, "4": {"no": "4", "mcq": "What allows a subclass to inherit properties and behaviors from a parent class?", "options": {"A": "Polymorphism", "B": "Encapsulation", "C": "Abstraction", "D": "Inheritance"}, "correct": "D"}, "5": {"no": "5", 

In [222]:
reviewed_quiz = response.get('reviewed_quiz')
reviewed_quiz

'{"1": {"no": "1", "mcq": "What is the main focus of Object-Oriented Programming?", "options": {"A": "Logic and functions", "B": "Objects and their interactions", "C": "Data structures and algorithms", "D": "Procedural programming"}, "correct": "B"}, "2": {"no": "2", "mcq": "What does a class define in OOP?", "options": {"A": "A specific object with its own data", "B": "Properties and methods for objects created from it", "C": "A set of unrelated functions", "D": "A collection of abstract data types"}, "correct": "B"}, "3": {"no": "3", "mcq": "Which OOP concept involves hiding implementation details and showing only essential features?", "options": {"A": "Encapsulation", "B": "Inheritance", "C": "Polymorphism", "D": "Abstraction"}, "correct": "A"}, "4": {"no": "4", "mcq": "What allows a subclass to inherit properties and behaviors from a parent class?", "options": {"A": "Polymorphism", "B": "Encapsulation", "C": "Abstraction", "D": "Inheritance"}, "correct": "D"}, "5": {"no": "5", "mcq

### Format the Reviewed Quiz and Store in DataFrame

In [223]:
import json
import pandas as pd

# 1. Get the reviewed_quiz JSON string from your response dict:
reviewed_quiz = response.get('reviewed_quiz')

# 2. Strip any markdown/code-block markers:
if reviewed_quiz.startswith("```json"):
    reviewed_quiz = reviewed_quiz.strip("```json").strip("```").strip()

# 3. Parse it into a Python list of dicts:
parsed_review = json.loads(reviewed_quiz)

In [224]:
parsed_review

{'1': {'no': '1',
  'mcq': 'What is the main focus of Object-Oriented Programming?',
  'options': {'A': 'Logic and functions',
   'B': 'Objects and their interactions',
   'C': 'Data structures and algorithms',
   'D': 'Procedural programming'},
  'correct': 'B'},
 '2': {'no': '2',
  'mcq': 'What does a class define in OOP?',
  'options': {'A': 'A specific object with its own data',
   'B': 'Properties and methods for objects created from it',
   'C': 'A set of unrelated functions',
   'D': 'A collection of abstract data types'},
  'correct': 'B'},
 '3': {'no': '3',
  'mcq': 'Which OOP concept involves hiding implementation details and showing only essential features?',
  'options': {'A': 'Encapsulation',
   'B': 'Inheritance',
   'C': 'Polymorphism',
   'D': 'Abstraction'},
  'correct': 'A'},
 '4': {'no': '4',
  'mcq': 'What allows a subclass to inherit properties and behaviors from a parent class?',
  'options': {'A': 'Polymorphism',
   'B': 'Encapsulation',
   'C': 'Abstraction',
  

In [238]:
df = pd.DataFrame.from_dict(parsed_review, orient='index')

# Drop the 'no' column


# Flatten the 'options' dictionary
options_df = df['options'].apply(pd.Series)

# Drop the 'options' column and concatenate options + correct at the end
df = df.drop(columns='options').join(options_df)

# Rename 'mcq' to 'Question'
df.rename(columns={'mcq': 'Question'}, inplace=True)
# Move 'correct' column to the end
correct = df.pop('correct')
df['correct'] = correct

# Optional: reset index, remove no.
df.reset_index(drop=True, inplace=True)
df.drop(columns='no', inplace=True)

In [239]:
df.head()

,Question,A,B,C,D,correct
0,What is the main focus of Object-Oriented Prog...,Logic and functions,Objects and their interactions,Data structures and algorithms,Procedural programming,B
1,What does a class define in OOP?,A specific object with its own data,Properties and methods for objects created fro...,A set of unrelated functions,A collection of abstract data types,B
2,Which OOP concept involves hiding implementati...,Encapsulation,Inheritance,Polymorphism,Abstraction,A
3,What allows a subclass to inherit properties a...,Polymorphism,Encapsulation,Abstraction,Inheritance,D
4,Which concept lets objects of different classe...,Encapsulation,Abstraction,Inheritance,Polymorphism,D
